In [ ]:
########################################################################################################################
# Filename: BPMLL_paragraph_classification.ipynb
#
# Purpose: Multi-label Text-categorization, using neural networks, for paragraph-level
#          data as part of STAT 6500 final project.

# Author(s): Bobby (Robert) Lumpkin, Yue Li
#
# Library Dependencies: numpy, pandas, tensorflow, bpmll
########################################################################################################################

# Text Categorization With Neural Networks 

In [74]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from bpmll import bp_mll_loss
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sys
sys.path.append('../ThresholdFunctionLearning')    ## Append path to the ThresholdFunctionLearning directory to the interpreters
                                                   ## search path
from threshold_learning import predict_test_labels_binary    ## Import the 'predict_test_labels_binary()' function from the 
                                                             ## threshold_learning library

## Load & Pre-Process the Data

In [5]:
## Load 'content_paragraphs_ready.csv' into a pandas dataframe
data_filepath = "..\..\dataset\content_paragraphs_ready.csv"
paragraph_data = pd.read_csv(data_filepath)
paragraph_data.head()
#os.getcwd()

,para_id,full_text,threats/impacts,responses/actions,severity,susceptibility,self-efficacy,external-efficacy,response efficacy,public health,...,prosper,preview,moor,coverag,glow,profil,clash,incumb,frequent,unfound
0,214236,MURPHY: Again Martha we are defacto staying at...,1,1,0,1,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,214232,"GOV. PHIL MURPHY, (D-NJ): Yes. Good to be back...",1,1,1,1,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,214266,"BEAUMONT (ON SCREEN UPPER LEFT - ""FRIDAY MARCH...",0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,214246,"But in the meantime, my message to Louisiana i...",1,1,1,0,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,214238,"MURPHY: Yeah listen, we had gotten another shi...",0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
## Keep only 'doc_id', the label columns, and the tf-idf columns
tfidf_colnames = list(paragraph_data.columns[25:])
label_columns = list(paragraph_data.columns[2:15])
cols_toKeep = ['doc_id']
cols_toKeep.extend(tfidf_colnames)
cols_toKeep.extend(label_columns)
paragraph_data = paragraph_data[cols_toKeep]
paragraph_data.head()

,doc_id,murphi,martha,defacto,stay,home,state,million,us,you�,...,susceptibility,self-efficacy,external-efficacy,response efficacy,public health,economy,education,political evaluation,racial conflict,international ralations/foreign policies
0,text1,1.684247,1.348455,2.161368,3.118616,3.016311,0.91833,1.207125,1.383217,1.763428,...,1,1,0,1,1,0,0,0,0,0
1,text2,1.684247,1.348455,0.000000,0.000000,0.000000,3.67332,0.000000,0.000000,1.763428,...,1,1,0,1,1,0,0,0,0,0
2,text3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0,0,1,0,1,0,0,0,0,0
3,text4,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0,1,0,1,1,0,0,0,0,0
4,text5,1.684247,1.348455,0.000000,0.000000,0.000000,0.00000,0.000000,1.383217,0.000000,...,0,0,1,0,1,0,0,0,0,0


In [65]:
## Define the X and Y train and test matrices
X = paragraph_data[tfidf_colnames].to_numpy().astype(float)
Y = paragraph_data[label_columns].to_numpy().astype(float)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33)

In [95]:
## These are the rows that don't have any associated labels
Y_gz = Y > 0
np.where(~Y_gz.any(axis=1))[0]

array([ 38,  58,  61,  67,  83, 132, 139, 142, 143, 150, 151, 166, 167,
       180, 214, 221, 240, 279, 282, 283, 284, 285, 286, 287, 288, 289],
      dtype=int64)

## Define and Train a BP-MLL Network

In [66]:
## Start with bp-mll loss
tf.random.set_seed(123)
num_labels = len(label_columns)

model_bpmll = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_labels, activation = 'sigmoid')
])

optim_func_bpmll = tf.keras.optimizers.Adagrad(
    learning_rate = 0.001, initial_accumulator_value = 0.1, epsilon = 1e-07,
    name = 'Adagrad')

model_bpmll.compile(optimizer = optim_func_bpmll,
              loss = bp_mll_loss,
              )

In [67]:
model_bpmll.fit(X_train, Y_train, epochs = 500)

Epoch 1/500
7/7 [==============================] - 1s 1ms/step - loss: nan
Epoch 2/500
7/7 [==============================] - 0s 1ms/step - loss: nan
Epoch 3/500
7/7 [==============================] - 0s 2ms/step - loss: nan
Epoch 4/500
7/7 [==============================] - 0s 2ms/step - loss: nan
Epoch 5/500
7/7 [==============================] - 0s 2ms/step - loss: nan
Epoch 6/500
7/7 [==============================] - 0s 2ms/step - loss: nan
Epoch 7/500
7/7 [==============================] - 0s 2ms/step - loss: nan
Epoch 8/500
7/7 [==============================] - 0s 1ms/step - loss: nan
Epoch 9/500
7/7 [==============================] - 0s 1ms/step - loss: nan
Epoch 10/500
7/7 [==============================] - 0s 2ms/step - loss: nan
Epoch 11/500
7/7 [==============================] - 0s 2ms/step - loss: nan
Epoch 12/500
7/7 [==============================] - 0s 1ms/step - loss: nan
Epoch 13/500
7/7 [==============================] - 0s 1ms/step - loss: nan
Epoch 14/500
7/7 [===

## Define and Train a Cross-Entropy Loss Network

In [70]:
## Start with standard cross-entropy loss (bpmll used later)

model_ce = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_labels, activation = 'sigmoid')
])

optim_func_ce = tf.keras.optimizers.Adagrad(
    learning_rate = 0.001, initial_accumulator_value = 0.1, epsilon = 1e-07,
    name = 'Adagrad')

model_ce.compile(optimizer = optim_func_ce,
              loss = 'categorical_crossentropy',
              )

In [71]:
model_ce.fit(X_train, Y_train, epochs = 500)

Epoch 1/500
7/7 [==============================] - 0s 2ms/step - loss: 8.3635
Epoch 2/500
7/7 [==============================] - 0s 2ms/step - loss: 8.2923
Epoch 3/500
7/7 [==============================] - 0s 2ms/step - loss: 8.5247
Epoch 4/500
7/7 [==============================] - 0s 2ms/step - loss: 8.0491
Epoch 5/500
7/7 [==============================] - 0s 1ms/step - loss: 8.0243
Epoch 6/500
7/7 [==============================] - 0s 1ms/step - loss: 8.0385
Epoch 7/500
7/7 [==============================] - 0s 2ms/step - loss: 8.1809
Epoch 8/500
7/7 [==============================] - 0s 1ms/step - loss: 8.1476
Epoch 9/500
7/7 [==============================] - 0s 1ms/step - loss: 8.0362
Epoch 10/500
7/7 [==============================] - 0s 1ms/step - loss: 7.9936
Epoch 11/500
7/7 [==============================] - 0s 937us/step - loss: 7.9305
Epoch 12/500
7/7 [==============================] - 0s 2ms/step - loss: 8.0398
Epoch 13/500
7/7 [==============================] - 0s 967u

## Compare Performance Using Different Threshold Function Approaches

In [75]:
## Using a constant 0.5 threshold function, get the hamming loss for the trained network on the test set
predictions = model_ce.predict(X_test)
predictions_binary = model_ce.predict(X_test)
for i in range(Y_test.shape[0]):
    for j in range(Y_test.shape[1]):
        if predictions_binary[i, j] > 0.5:
            predictions_binary[i, j] = 1
        else:
            predictions_binary[i, j] = 0

# Get the hamming loss
metrics.hamming_loss(Y_test, predictions_binary)

0.5913461538461539

In [76]:
## Learn a Threshold Function
Y_train_pred = model_ce.predict(X_train)
Y_test_pred = model_ce.predict(X_test)
t_range = (0, 1)

test_labels_binary, threshold_function = predict_test_labels_binary(Y_train_pred, Y_train, Y_test_pred, t_range)
metrics.hamming_loss(Y_test, test_labels_binary)

0.1875

In [90]:
## Compare predicted test labels for instance against true test labels in next cell
test_labels_binary[5,]

array([0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.])

In [91]:
## Compare true test labels for instane against predicted test labels in previous cell
Y_test[5,]

array([0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0.])

In [98]:
## Compute the proportion of labels that are equal to one in the test set
## This tells us what the hamming loss would be, if we constantly predicted 0
## We can compare our observed hamming loss against this
np.sum(Y_test == 1) / (Y_test.shape[0] * Y_test.shape[1])

0.21474358974358973